ToDo:
    - Georeferencing
    - Tests
    - Fehlermedlung der API einbauen
    - 

In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import geopandas as gpd
import getpass
import xarray
import rasterio as rio
import os
import pandas as pd
import numpy as np
import shutil
from time import sleep
import stat
import io

directory = 'C:/Users/adria/Desktop/Uni/Semester5/Geosoft2/Code/Notebooks/Sentinel/Data'
resolution = 60    #10, 20 ,60 möglich

In [2]:
def downloadingData (aoi, collectionDate, plName, prLevel, clouds):
    '''
    Downloads the Sentinel2-Data with the given parameters
    
    Parameter: 
        aoi(str) = The type and the coordinates of the area of interest
        collectionDate = The date of the data
        plName(str) = The name of the platform
        prLevel(str) = The name of the process
        clouds = The allowed percentage of the cloudcover
    '''
    
    '''Loggin in the Scihub - API with username and password'''
    username = getpass.getpass("user: ")
    password = getpass.getpass("password: ")
    api = SentinelAPI(username, password, 'https://scihub.copernicus.eu/dhus')
    
    '''Choosing the data with bounding box (footprint), date, platformname, processinglevel and cloudcoverpercentage'''
    products = api.query(aoi, date = collectionDate, platformname = plName, processinglevel = prLevel, cloudcoverpercentage = clouds)
    
    '''Filters the products and sort the by cloudcoverpercentage'''
    products_gdf = api.to_geodataframe(products)
    products_gdf_sorted = products_gdf.sort_values(['cloudcoverpercentage'], ascending=[True])
    
    '''Downloads the choosen files from Scihub'''
    #saveFile(products_gdf_sorted)

    products_gdf_sorted.to_csv('w')
    api.download_all(products,directory, max_attempts=10, checksum=True)

In [3]:
def listDir (path):
    '''
    Lists all files from the given directory
    
    Parameter: 
        path(str): Path to the directory
        
    Returns:
        path(str[]): An array of all filenames
    '''
    return os.listdir(path)

In [4]:
from zipfile import ZipFile
def unzip (filename):
    '''
    Unzips the file with the given filename
    
    Parameter: 
        filename(str): Name of the .zip file
    '''
    with ZipFile(os.path.join(directory, filename), 'r') as zipObj:
        zipObj.extractall(directory)

In [5]:
def delete (path):
    '''
    Deletes the file/directory with the given path
    
    Parameter: 
        path(str): Path to the file/directory
    '''
    if os.path.exists(path):
        os.remove(path)
        print("File deleted: " + path)
    else:
        print("The file does not exist")

In [6]:
def extractBands (filename,resolution):
    '''
    Extracts bandpaths from .SAFE file
    
    Parameter: 
        filename(str): Sentinel .SAFE file
        
    Returns:
        bandPaths(str[]): An array of the paths for the red and nir band
    '''
    lTwoA = listDir(os.path.join(directory, filename, "GRANULE"))
    
    if resolution == 60:
        bandName = listDir (os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R60m"))
        pathRed = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R60m", str(bandName[4]))
        pathNIR = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R60m", str(bandName[11]))
        bandPaths = [pathRed, pathNIR]
    
    elif resolution == 20:   
        bandName = listDir (os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R20m"))
        pathRed = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R20m", str(bandName[3]))
        pathNIR = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R20m", str(bandName[9]))
        bandPaths = [pathRed, pathNIR]
    
    elif resolution == 10:
        bandName = listDir (os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R10m"))
        pathRed = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R10m", str(bandName[3]))
        pathNIR = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R10m", str(bandName[4]))
        bandPaths = [pathRed, pathNIR]
    else:
        print("no such resolution")
        return -1
    
    return bandPaths

In [7]:
def loadBand (bandpath,date,tile,resolution):
    '''
    Opens and reads the red and nir band, saves them as NetCDF file
    
    Parameter: 
        bandPaths(str[]): Array with the paths to the red and nir band
    '''
    b4 = rio.open(bandpath[0])
    b8 = rio.open(bandpath[1])
    red = b4.read()
    nir = b8.read()    
    
    print(b8.bounds)
    print(b8.crs)

    if resolution==10:
        res=1830*3*2
    elif resolution == 20:
        res = 1830*3
    elif resolution == 60:
        res = 1830
    else:
        print("No such resolution")
        return -1
    
    j=res-1
    i=0
    lat = [0]*res
    lon = [0]*res
    while j>=0:
        lon[i]=b4.bounds.left + i*60
        lat[i]=b4.bounds.bottom + j*60
        i=i+1
        j=j-1


    time = pd.date_range(date, periods=1)
    
    dataset = xarray.Dataset(
          {
        "red": (["time","lat", "lon"], red),
        "nir": (["time","lat", "lon"], nir)
        },
         coords=dict( 
            time=time,
            lat=(["lat"], lat),
            lon=(["lon"], lon),
        ),
         attrs=dict(       
             platform= plName,
             processingLevel= prLevel,
             cloudcover = clouds,
             source = "https://scihub.copernicus.eu/dhus"
         ),
    )
  
    print(dataset)

    dataset.to_netcdf(directory+"/Data"+"datacube_"+str(date)+"_"+str(tile)+"_R"+str(resolution)+".nc", 'w', format='NETCDF4')
    b4.close()
    b8.close()
    return dataset

In [8]:
def getDate(filename):
    '''
    extracts the Date of the Sentinelfilename
    Parameters:
        filename (str): name of the file
    Returns: 
        (str): Date of the File ("2020-12-31")
    '''
    return filename[11:15]+"-"+filename[15:17]+"-"+filename[17:19]

In [9]:
def getTile(filename):
    '''
    extracts the UTM-tile of the Sentinelfilename
    Parameters:
        filename (str): name of the file
    Returns: 
        (str): UTM-tile of the File ("31UMC")
    '''
    return filename[38:44]

In [10]:
def on_rm_error( func, path, exc_info):
       # path contains the path of the file that couldn't be removed
       # let's just assume that it's read-only and unlink it.
       os.chmod( path, stat.S_IWRITE )
       os.unlink( path )


Execution methods

In [19]:
'''Parameters for the download'''
aoi = 'POLYGON((7.181324516246491 52.15109039913986,8.04022788336513 52.151774818649216,8.043574260120138 51.69984086555013,7.166823550308125 51.70675374223106,7.181324516246491 52.15109039913986,7.181324516246491 52.15109039913986))'
collectionDate = ('20200601', '20200605')
plName = 'Sentinel-2'
prLevel = 'Level-2A'
clouds = (0,30)

In [17]:
'''Downloading the data'''
downloadingData (aoi, collectionDate, plName, prLevel, clouds)

user: ········
password: ········


C:\Users\adria\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
Downloading:   0%|                                                                          | 0.00/416M [00:00<?, ?B/s]
Downloading:   0%|                                                                          | 0.00/289M [00:00<?, ?B/s]
Downloading:   0%|▏                                                                | 1.05M/289M [00:00<01:57, 2.44MB/s]
Downloading:   0%|▏                                                                | 1.05M/416M [00:00<05:37, 1.23MB/s]
Downloading:   1%|▋                                                                | 3.15M/289M [00:00<01:28, 3.2

Downloading:  13%|████████▋                                                        | 55.6M/416M [00:21<01:41, 3.53MB/s]
Downloading:  14%|████████▊                                                        | 56.6M/416M [00:21<01:41, 3.55MB/s]
Downloading:  14%|█████████                                                        | 57.7M/416M [00:22<01:47, 3.34MB/s]
Downloading:  14%|█████████▎                                                       | 59.8M/416M [00:22<01:49, 3.26MB/s]
Downloading:  15%|█████████▌                                                       | 60.8M/416M [00:22<01:48, 3.28MB/s]
Downloading:  15%|█████████▋                                                       | 61.9M/416M [00:23<01:51, 3.18MB/s]
Downloading:  15%|█████████▊                                                       | 62.9M/416M [00:23<01:49, 3.21MB/s]
Downloading:  16%|██████████▏                                                      | 65.0M/416M [00:24<01:48, 3.24MB/s]
Downloading:  16%|██████████▎           

Downloading:  36%|███████████████████████▊                                          | 150M/416M [00:47<01:02, 4.29MB/s]
Downloading:  37%|████████████████████████                                          | 152M/416M [00:48<00:59, 4.42MB/s]
Downloading:  37%|████████████████████████▍                                         | 154M/416M [00:48<00:58, 4.46MB/s]
Downloading:  38%|████████████████████████▊                                         | 156M/416M [00:49<00:59, 4.39MB/s]
Downloading:  38%|█████████████████████████                                         | 158M/416M [00:49<01:05, 3.96MB/s]
Downloading:  39%|█████████████████████████▍                                        | 160M/416M [00:50<01:00, 4.20MB/s]
Downloading:  39%|█████████████████████████▊                                        | 163M/416M [00:50<00:59, 4.26MB/s]
Downloading:  40%|██████████████████████████                                        | 165M/416M [00:51<00:58, 4.27MB/s]
Downloading:  40%|██████████████████████

Downloading:  70%|██████████████████████████████████████████████▍                   | 203M/289M [01:16<00:30, 2.82MB/s]
Downloading:  65%|██████████████████████████████████████████▊                       | 269M/416M [01:17<00:42, 3.41MB/s]
Downloading:  65%|██████████████████████████████████████████▉                       | 271M/416M [01:17<00:44, 3.30MB/s]
Downloading:  65%|███████████████████████████████████████████                       | 272M/416M [01:17<00:43, 3.32MB/s]
Downloading:  66%|███████████████████████████████████████████▎                      | 273M/416M [01:18<00:43, 3.30MB/s]
Downloading:  66%|███████████████████████████████████████████▌                      | 275M/416M [01:18<00:40, 3.50MB/s]
Downloading:  66%|███████████████████████████████████████████▊                      | 276M/416M [01:19<00:39, 3.56MB/s]
Downloading:  67%|████████████████████████████████████████████                      | 278M/416M [01:19<00:37, 3.70MB/s]
Downloading:  67%|██████████████████████

...

Downloading:  80%|████████████████████████████████████████████████████▏            | 969M/1.21G [05:33<01:35, 2.48MB/s]
Downloading:  80%|████████████████████████████████████████████████████▎            | 971M/1.21G [05:33<01:29, 2.64MB/s]
Downloading:  81%|████████████████████████████████████████████████████▍            | 973M/1.21G [05:34<01:38, 2.37MB/s]
Downloading:  81%|████████████████████████████████████████████████████▍            | 974M/1.21G [05:35<01:39, 2.35MB/s]
Downloading:  81%|████████████████████████████████████████████████████▌            | 976M/1.21G [05:36<01:42, 2.25MB/s]
Downloading:  81%|████████████████████████████████████████████████████▋            | 978M/1.21G [05:37<01:43, 2.21MB/s]
Downloading:  81%|████████████████████████████████████████████████████▋            | 979M/1.21G [05:37<01:44, 2.17MB/s]
Downloading:  81%|████████████████████████████████████████████████████▊            | 981M/1.21G [05:38<01:48, 2.08MB/s]
Downloading:  81%|██████████████████████

Downloading:  87%|███████████████████████████████████████████████████████▌        | 1.05G/1.21G [06:21<01:37, 1.63MB/s]
Downloading:  87%|███████████████████████████████████████████████████████▋        | 1.05G/1.21G [06:22<01:33, 1.68MB/s]
Downloading:  87%|███████████████████████████████████████████████████████▋        | 1.05G/1.21G [06:22<01:31, 1.71MB/s]
Downloading:  87%|███████████████████████████████████████████████████████▊        | 1.05G/1.21G [06:23<01:28, 1.75MB/s]
Downloading:  81%|████████████████████████████████████████████████████▋            | 943M/1.16G [06:38<01:43, 2.12MB/s]
Downloading:  87%|███████████████████████████████████████████████████████▊        | 1.05G/1.21G [06:23<01:25, 1.82MB/s]
Downloading:  87%|███████████████████████████████████████████████████████▊        | 1.05G/1.21G [06:24<01:26, 1.77MB/s]
Downloading:  87%|███████████████████████████████████████████████████████▉        | 1.05G/1.21G [06:25<01:22, 1.85MB/s]
Downloading:  87%|██████████████████████

Downloading:  87%|███████████████████████████████████████████████████████▍        | 1.01G/1.16G [07:13<01:21, 1.89MB/s]
Downloading:  92%|██████████████████████████████████████████████████████████▊     | 1.11G/1.21G [06:59<01:06, 1.46MB/s]
Downloading:  92%|██████████████████████████████████████████████████████████▉     | 1.11G/1.21G [06:59<01:05, 1.47MB/s]
Downloading:  87%|███████████████████████████████████████████████████████▋        | 1.01G/1.16G [07:14<01:15, 2.00MB/s]
Downloading:  92%|██████████████████████████████████████████████████████████▉     | 1.11G/1.21G [07:00<01:13, 1.30MB/s]
Downloading:  87%|███████████████████████████████████████████████████████▊        | 1.01G/1.16G [07:16<01:22, 1.80MB/s]
Downloading:  92%|███████████████████████████████████████████████████████████     | 1.11G/1.21G [07:01<01:17, 1.21MB/s]
Downloading:  92%|███████████████████████████████████████████████████████████     | 1.11G/1.21G [07:02<01:14, 1.25MB/s]
Downloading:  87%|██████████████████████

Downloading:  98%|██████████████████████████████████████████████████████████████▍ | 1.18G/1.21G [07:39<00:15, 1.84MB/s]
Downloading:  98%|██████████████████████████████████████████████████████████████▌ | 1.18G/1.21G [07:40<00:15, 1.73MB/s]
Downloading:  98%|██████████████████████████████████████████████████████████████▌ | 1.18G/1.21G [07:41<00:14, 1.77MB/s]
Downloading:  98%|██████████████████████████████████████████████████████████████▋ | 1.18G/1.21G [07:41<00:15, 1.68MB/s]
Downloading:  98%|██████████████████████████████████████████████████████████████▋ | 1.18G/1.21G [07:42<00:14, 1.69MB/s]
Downloading:  98%|██████████████████████████████████████████████████████████████▊ | 1.18G/1.21G [07:43<00:13, 1.71MB/s]
Downloading:  98%|██████████████████████████████████████████████████████████████▊ | 1.18G/1.21G [07:43<00:12, 1.72MB/s]
Downloading:  98%|██████████████████████████████████████████████████████████████▊ | 1.19G/1.21G [07:44<00:12, 1.76MB/s]
Downloading:  98%|██████████████████████

Downloading:  99%|███████████████████████████████████████████████████████████████ | 1.15G/1.16G [08:25<00:07, 2.35MB/s]
Downloading:   2%|█                                                               | 21.0M/1.21G [00:09<09:36, 2.05MB/s]
Downloading:   2%|█▏                                                              | 22.0M/1.21G [00:10<09:20, 2.11MB/s]
Downloading:   2%|█▏                                                              | 23.1M/1.21G [00:10<09:29, 2.08MB/s]
Downloading:  99%|███████████████████████████████████████████████████████████████▎| 1.15G/1.16G [08:27<00:04, 2.69MB/s]
Downloading:   2%|█▎                                                              | 24.1M/1.21G [00:11<09:05, 2.17MB/s]
Downloading:   2%|█▎                                                              | 25.2M/1.21G [00:11<09:26, 2.08MB/s]
Downloading:   2%|█▍                                                              | 26.2M/1.21G [00:12<08:59, 2.18MB/s]
Downloading:   2%|█▍                    

In [12]:
'''Unzips all data and deletes .zip file'''
for filename in os.listdir(directory):
    if filename.endswith(".zip"): 
        unzip(filename)
        delete(os.path.join(directory, filename))
        continue
    else:
        continue

File deleted: C:/Users/adria/Desktop/Uni/Semester5/Geosoft2/Code/Notebooks/Sentinel/Data\S2B_MSIL2A_20200601T103629_N0214_R008_T31UGT_20200601T135554.zip
File deleted: C:/Users/adria/Desktop/Uni/Semester5/Geosoft2/Code/Notebooks/Sentinel/Data\S2B_MSIL2A_20200601T103629_N0214_R008_T32ULC_20200601T135554.zip
File deleted: C:/Users/adria/Desktop/Uni/Semester5/Geosoft2/Code/Notebooks/Sentinel/Data\S2B_MSIL2A_20200601T103629_N0214_R008_T32UMC_20200601T135554.zip


In [14]:
'''Works with .SAFE data'''
for filename in os.listdir(directory):
    if filename.endswith(".SAFE"): 
        bandPath = extractBands(os.path.join(directory, filename),resolution)
        loadBand(bandPath,getDate(filename),getTile(filename),resolution)
        sleep(0.5)
#         shutil.rmtree(os.path.join(directory, filename) , onerror = on_rm_error ) #delete safe datei
        continue
    else:
        continue

1830
1830
BoundingBox(left=699960.0, bottom=5690220.0, right=809760.0, top=5800020.0)
5690220.0
5800020.0
699960.0
809760.0
| 60.00, 0.00, 699960.00|
| 0.00,-60.00, 5800020.00|
| 0.00, 0.00, 1.00|
EPSG:32631
(699960.0, 5800020.0)
(809760.0, 5690220.0)
(1,)
(1830, 1830)
1830
1830
<xarray.Dataset>
Dimensions:  (lat: 1830, lon: 1830, time: 1)
Coordinates:
  * time     (time) datetime64[ns] 2020-06-01
  * lat      (lat) float64 5.8e+06 5.8e+06 5.8e+06 ... 5.69e+06 5.69e+06
  * lon      (lon) float64 7e+05 7e+05 7.001e+05 ... 8.096e+05 8.097e+05
Data variables:
    red      (time, lat, lon) uint16 266 325 308 310 312 ... 454 519 622 608 424
    nir      (time, lat, lon) uint16 3443 2548 2589 2598 ... 3924 3693 3755 3835
Attributes:
    platform:         Sentinel-2
    processingLevel:  Level-2A
    cloudcover:       (0, 90)
    source:           https://scihub.copernicus.eu/dhus
1830
1830
BoundingBox(left=300000.0, bottom=5690220.0, right=409800.0, top=5800020.0)
5690220.0
5800020.0
300000.